# SD202 TP2 - Normalization and SQL

The objectives of this TP are the following:

1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this lab, we are going to use a database containing wine information related to 'production' and 'sales'. 

Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


We need to prepare the SQL environment:

In [2]:
import sqlite3

In [3]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [4]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
Keys1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
Keys2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Lieu2:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producteurs1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Producteurs2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
WINES:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wines1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wines2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


We recommend inline __%sql__ as an alternative to sqlite3 package

In [5]:
%load_ext sql
%sql sqlite:///wine.db

Now, we can see the content of the tables using SQL queries:

In [6]:
%sql SELECT DISTINCT NV, CRU, MILL, DEGRE FROM MASTER1;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
None,None,None,None
1,Mercurey,1980,11.5
2,Julienas,1974,11.3
3,Savigny les Beaunes,1978,12.1
4,Mercurey,1980,10.9
5,Pommard,1976,11.7
6,Mercurey,1981,11.2
7,Grands Echezeaux,1968,11.7
8,Cotes de Beaune Villages,1975,12.3
9,Chapelle Chambertin,1973,11.9


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables Master1 and Master2 are in the First Normal Form (1NF) and suffer from data redundancy. 

__1.1__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE

Convert table Master1 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.

* Explain your answer
* Describe the schema of new tables and how they relate

**Table 1 - NV to CRU DEGRE MILL - MASTER1**

In [7]:
%sql SELECT DISTINCT NV, CRU, DEGRE, MILL FROM MASTER1 WHERE NV IS NOT NULL LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


**Table 2 - NP to NOM PRENOM REGION - MASTER1**

In [8]:
%sql SELECT DISTINCT NP, NOM, PRENOM, REGION FROM MASTER1 WHERE NP IS NOT NULL LIMIT 10;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


**Table 3 - NP <-> NV Correspondence, to QTE - MASTER1**

In [9]:
%%sql SELECT DISTINCT NP, NV, QTE FROM MASTER1 
WHERE NP IS NOT NULL
AND NV IS NOT NULL 
LIMIT 10;

 * sqlite:///wine.db
Done.


NP,NV,QTE
1,1,300
73,1,1
5,2,100
1,3,400
10,4,35
30,5,46
42,6,300
98,7,60
90,8,12
98,10,100


The keys of the tables are pretty clear : NV is the foreign key of table 1, NP is the foreign key of table 2, with both primary keys being in table 3, which primary key is the duo NP NV. 
</br> This relation is at least 2NF because there's only one key for tables 1 and 2 and for table 3 you need both NP and NB to get QTE as for instance for NV = 1 you can get QTE = 300 or QTE = 1. Same for NV = 64 for instance (you can check some more with the request : 

In [10]:
"""%%sql SELECT DISTINCT NP, NV, QTE FROM MASTER1 
WHERE NP IS NOT NULL
AND 
(NV IS NOT NULL AND NV IS NOT 1)
LIMIT 10;"""

'%%sql SELECT DISTINCT NP, NV, QTE FROM MASTER1 \nWHERE NP IS NOT NULL\nAND \n(NV IS NOT NULL AND NV IS NOT 1)\nLIMIT 10;'

And as the table is indeed 1NF, it's also 2NF because of the 'the whole key' argument : you need the whole key in all tables to identify the rest of the columns. </br>
Now during the original time of the TP I was told this relation was not 3NF. As table 1 is 3NF, I only see two reasons for this relation not to be 3NF : all NOM are different which allows to find PRENOM through NOM and not only the key, which would make this relation 2NF only. But I didn't find a counterexample. The other reason would be that there's redundancy in NP w.r.t. NV in table 3, which was what I was told last tuesday, but I haven't exactly found that this prevented the relation from being 3NF. 3NF, the 'only the key' argument, states that you should not be able to find any dependency that doesn't imply the key. That is, only dependencies from the key are allowed. Which is what we have in table 1 because of how it's built, in table 3 because there's only the key + one other column (and as we saw you need both NP and NV to identify QTE, both NP and NV have repetitions), and **maybe** in table 2 as I explained earlier. Otherwise it's 3NF, we split the data well enough !

__1.2__ Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION

Convert table Master2 to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable. 
* Explain your answer
* Describe the schema of new tables and how they relate

**Table 1 - NV to CRU DEGRE MILL - MASTER2**

In [11]:
%sql SELECT DISTINCT NV, CRU, DEGRE, MILL FROM MASTER2 WHERE NV IS NOT NULL LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


**Table 2 - NB to NOM PRENOM TYPE - MASTER2**

In [12]:
%sql SELECT NB, NOM, PRENOM, TYPE FROM MASTER2 WHERE NB IS NOT NULL LIMIT 10;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


**Table 3 - NV, NB to DATES, LIEU, QTE - MASTER2**

In [13]:
%%sql SELECT NV, NB, DATES, LIEU, QTE FROM MASTER2
WHERE NV IS NOT NULL
AND NB IS NOT NULL
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE
1,2,1977-11-02,BORDEAUX,33
1,44,2015-10-16,PARIS,1
1,45,1983-12-31,RENNES,1
2,48,1983-12-25,LYON,2
3,7,1978-11-01,NICE,6
3,49,1987-12-24,MERCUREY,5
4,8,1982-11-05,TOULOUSE,12
4,44,2015-10-16,PARIS,6
4,50,1981-06-29,SENS,4
5,2,1984-11-04,MERCUREY,50


**Table 4 - LIEU to REGION - MASTER2**

In [14]:
%sql SELECT LIEU, REGION FROM MASTER2 WHERE REGION IS NOT NULL LIMIT 10;

 * sqlite:///wine.db
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
PARIS,ÎLE-DE-FRANCE
RENNES,BRETAGNE
LYON,AUVERGNE-RHÔNE-ALPES
NICE,PROVENCE-ALPES-CÔTE D'AZUR
MERCUREY,BOURGOGNE-FRANCHE-COMTÉ
TOULOUSE,OCCITANIE
PARIS,ÎLE-DE-FRANCE
SENS,BOURGOGNE-FRANCHE-COMTÉ
MERCUREY,BOURGOGNE-FRANCHE-COMTÉ


So, again, same as before : the keys are NV, NB, NV-NB, LIEU. It's at least 2NF ('the-whole-key' mode) because : for tables 1-2-4 the key is only one column, for table 3 there are enough redundancies in both NB and NV so that both are needed to make a key. Is it 3NF? Is it 'only the key' ? The answer is no, because in table 2, you can indentify TYPE through PRENOM + NAME, without using NB. Maybe even in practice, you can identify it through NAME if not two names are equal (but then PRENOM also becomes a non-3NF factor, and we made this assumption in Q1). You don't need the abstract key NB to identify that someone is a petit, medium or large drinker if you have their first and last name already ! We have to create a sub-table named 'Type' which has for foreign key PRENOM + NOM (in case two people have the same NOM or PRENOM) and as a value TYPE. This way there's no problem. 

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables. A table can be created from the result of a query. In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [15]:
%sql DROP TABLE IF EXISTS dummy;

# Create dummy table
%sql CREATE TABLE dummy AS \
SELECT DISTINCT DEGRE \
FROM MASTER1;

print("\nContent of the database")
printSchema(conn)

print("\nContent of dummy")
%sql SELECT * FROM dummy

 * sqlite:///wine.db
Done.
 * sqlite:///wine.db
Done.

Content of the database
Keys1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
Keys2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Lieu2:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producteurs1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Producteurs2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
WINES:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wines1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wines2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
dummy:
  0: DEGRE(NUM)

Content of dummy
 * sqlite:///wine

DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


In [16]:
# Remove dummy table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

__1.3__ Create the new tables from Master1:

In [17]:
%%sql 

DROP TABLE IF EXISTS Wines1;

CREATE TABLE Wines1 AS 
SELECT DISTINCT NV, CRU, DEGRE, MILL FROM MASTER1 WHERE NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [18]:
%%sql 

DROP TABLE IF EXISTS Producteurs1;

CREATE TABLE Producteurs1 AS
SELECT DISTINCT NP, NOM, PRENOM, REGION FROM MASTER1 WHERE NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [19]:
%%sql 

DROP TABLE IF EXISTS Keys1;

CREATE TABLE Keys1 AS 
SELECT DISTINCT NP, NV, QTE FROM MASTER1 WHERE NP IS NOT NULL AND NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

__1.4__ Create the new tables from Master2:

In [20]:
%%sql DROP TABLE IF EXISTS Wines2;

CREATE TABLE Wines2 AS 
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [21]:
%%sql DROP TABLE IF EXISTS Producteurs2;

CREATE TABLE Producteurs2 AS 
SELECT NB, NOM, PRENOM, TYPE FROM MASTER2 
WHERE NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [22]:
%%sql DROP TABLE IF EXISTS Keys2;

CREATE TABLE Keys2 AS 
SELECT NV, NB, DATES, LIEU, QTE FROM MASTER2
WHERE NV IS NOT NULL
AND NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [23]:
%%sql DROP TABLE IF EXISTS Lieu2;

CREATE TABLE Lieu2 AS 
SELECT LIEU, REGION FROM MASTER2 
WHERE LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [24]:
#No need to put WHERE NV IS NOT NULL because it's they key, it can't be NULL here. 
print("\nContent of the database")
printSchema(conn)

print("\nContent of WINES")
%sql SELECT * FROM WINES LIMIT 10;


Content of the database
Keys1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
Keys2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Lieu2:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producteurs1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Producteurs2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
WINES:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wines1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wines2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)

Content of WINES
 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

__2.1__ What are the different types of clients (buveurs) by volume of purchases?

First answer - show all types of clients for every volume of purchase

In [25]:
%%sql 
SELECT DISTINCT Producteurs2.TYPE AS TYPE, Keys2.QTE AS QTE 
FROM Keys2 JOIN Producteurs2 ON Keys2.NB = Producteurs2.NB 
WHERE QTE IS NOT NULL
ORDER BY QTE
LIMIT 15;

 * sqlite:///wine.db
Done.


TYPE,QTE
petit,1
gros,1
moyen,1
moyen,2
petit,2
gros,2
petit,3
gros,3
moyen,3
moyen,4


Second answer - show only possible answers, that is, what are the 3 types of buveurs that exist ? What are the different types of buveurs overall ? 

In [26]:
%%sql 
SELECT DISTINCT TYPE AS types_of_buveurs FROM Producteurs2;

 * sqlite:///wine.db
Done.


types_of_buveurs
petit
moyen
gros


__2.2__ What regions produce Pommard or Brouilly?

In [27]:
%%sql 
SELECT Wines1.CRU AS Cru, Producteurs1.REGION 
FROM Keys1 JOIN Producteurs1 ON Keys1.NP = Producteurs1.NP
JOIN Wines1 ON Keys1.NV = Wines1.NV
WHERE Cru == 'Pommard'
OR Cru == 'Brouilly';

 * sqlite:///wine.db
Done.


Cru,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


__2.3__ What regions produce Pommard and Brouilly?

In [28]:
%%sql 
SELECT Producteurs1.REGION 
FROM Keys1 JOIN Producteurs1 ON Keys1.NP = Producteurs1.NP
JOIN Wines1 ON Keys1.NV = Wines1.NV
WHERE CRU IN ('Pommard', 'Brouilly')
GROUP BY REGION
HAVING COUNT(DISTINCT CRU) = 2

 * sqlite:///wine.db
Done.


REGION
Bourgogne


__2.4__ Get the number of wines bought by CRU and Millésime

Please note that the exact same request can be made with tables created from Master1 instead of Master2, it just gives different numerical results but the table is the same. I just wanted the same ones as the default ones.

In [29]:
%%sql 
SELECT DISTINCT Wines2.CRU AS Cru, Wines2.MILL AS Mill, SUM(QTE) AS Number_of_wines_bought
FROM Keys2 JOIN Wines2 ON Keys2.NV = Wines2.NV 
GROUP BY Cru, Mill ;

 * sqlite:///wine.db
Done.


Cru,Mill,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


__2.5__ Retrieve the wine number (NV) of wines produced by more than three producers

In [30]:
%%sql
SELECT Keys1.NV, COUNT(*) AS number_of_producteurs
FROM Keys1
GROUP BY Keys1.NV
HAVING number_of_producteurs > 3

 * sqlite:///wine.db
Done.


NV,number_of_producteurs
45,5
78,5
89,4
98,5


__2.6__ Which producers have not produced any wine?

I first wanted to implement Keys1.QTE = 0 or = None, but producers simply don't appear in the Keys1 list when they haven't produced anything. So I've changed my code :

In [31]:
%%sql 
SELECT Producteurs1.NP , Producteurs1.NOM, Producteurs1.PRENOM 
FROM Producteurs1 
WHERE Producteurs1.NP NOT IN(SELECT Keys1.NP FROM Keys1)

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


__2.7__ What clients (buveurs) have bought at least one wine from 1980?

In [32]:
%%sql
SELECT DISTINCT Producteurs2.NB, Producteurs2.NOM, Producteurs2.PRENOM
FROM Keys2 JOIN Producteurs2 ON Keys2.NB = Producteurs2.NB
JOIN Wines2 ON Keys2.NV = Wines2.NV
WHERE Wines2.MILL = 1980
ORDER BY Keys2.NB

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


__2.8__ What clients (buveurs) have NOT bought any wine from 1980?

In [33]:
%%sql
SELECT DISTINCT Producteurs2.*
FROM Keys2 JOIN Producteurs2 ON Keys2.NB = Producteurs2.NB
WHERE Producteurs2.NB NOT IN 

(SELECT DISTINCT Producteurs2.NB
FROM Keys2 JOIN Producteurs2 ON Keys2.NB = Producteurs2.NB
JOIN Wines2 ON Keys2.NV = Wines2.NV
WHERE Wines2.MILL = 1980)
ORDER BY Keys2.NB

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
9,Ajar,Emile,petit
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
14,Bory,Jean Louis,gros


__2.9__ What clients (buveurs) have bought ONLY wines from 1980? (I figured out I'd just take all buyers of 1980 and remove those who bought from another year, which makes an easy except of both previous requests)

In [38]:
%%sql SELECT DISTINCT Producteurs2.NB, Producteurs2.NOM, Producteurs2.PRENOM
FROM Keys2 JOIN Wines2 ON Keys2.NV = Wines2.NV JOIN Producteurs2 ON Keys2.NB = Producteurs2.NB
WHERE Wines2.MILL = 1980
EXCEPT
SELECT DISTINCT Producteurs2.NB, Producteurs2.NOM, Producteurs2.PRENOM
FROM Keys2 JOIN Wines2 ON Keys2.NV = Wines2.NV JOIN Producteurs2 ON Keys2.NB = Producteurs2.NB
WHERE Wines2.MILL <> 1980

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


For some reason this query doesn't work. What I do is I take all producers which number of wines they bought is equal to the number of wines from 1980 they bought. But it doesn't yield anything...

__2.10__ List all wines from 1980

In [35]:
%%sql
SELECT Wines1.*
FROM Wines1
WHERE Wines1.MILL = 1980

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


__2.11__ What are the wines from 1980 bought by NB=2?

In [36]:
%%sql
SELECT Keys2.*, Wines2.*
FROM Keys2 JOIN Wines2 ON Keys2.NV = Wines2.NV
WHERE Wines2.MILL = 1980 AND Keys2.NB = 2

 * sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE,NV_1,CRU,DEGRE,MILL
1,2,1977-11-02,BORDEAUX,33,1,Mercurey,11.5,1980


__2.12__ What clients (buveurs) have bought ALL the wines from 1980?

In [70]:
%%sql SELECT DISTINCT Producteurs2.NB, Producteurs2.NOM, Producteurs2.PRENOM
FROM (SELECT COUNT(Keys2.NV) AS wineCounter, Keys2.NB
      FROM Keys2
      GROUP BY Keys2.NB

      INTERSECT
      
      SELECT wineCounter, Producteurs2.NB
      FROM (SELECT COUNT(Wines2.NV) AS wineCounter
            FROM Wines2
            WHERE Wines2.MILL=1980), Producteurs2
            ) AS Counter1980 JOIN Producteurs2 ON Counter1980.NB = Producteurs2.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre


I finally made it work (with some help of friends) - the idea I had was the correct one, about counting all wines of 1980 <strong>bought by someone</strong> (that's the GROUP BY call), and counting all the wines of 1980, I just didn't use the COUNT call (which was very stupid of me, because it's literally made to count things...)

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$<strong>Last updated (aside from our signature here) - 13/06/2022 - 23h16 - CAPITAN Guillaume & MOALIC Baptiste </strong>